In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn.model_selection import train_test_split
from sklearn.cluster import AgglomerativeClustering,KMeans
from sklearn.impute import KNNImputer
import time
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
import itertools
from pandas_profiling import ProfileReport

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/olympic-historical-dataset-from-olympediaorg/Olympic_Games_Medal_Tally.csv
/kaggle/input/olympic-historical-dataset-from-olympediaorg/Olympic_Athlete_Event_Results.csv
/kaggle/input/olympic-historical-dataset-from-olympediaorg/Olympics_Country.csv
/kaggle/input/olympic-historical-dataset-from-olympediaorg/Olympic_Results.csv
/kaggle/input/olympic-historical-dataset-from-olympediaorg/Olympics_Games.csv
/kaggle/input/olympic-historical-dataset-from-olympediaorg/Olympic_Athlete_Bio.csv


In [2]:
results_data = pd.read_csv('../input/olympic-historical-dataset-from-olympediaorg/Olympic_Athlete_Event_Results.csv')
athlete_data = pd.read_csv('../input/olympic-historical-dataset-from-olympediaorg/Olympic_Athlete_Bio.csv')
olympic_games = pd.read_csv('../input/olympic-historical-dataset-from-olympediaorg/Olympics_Games.csv')


In [3]:
#results_data.set_index('athlete_id', inplace=True)
#athlete_data.set_index('athlete_id', inplace=True)

athleteResults = athlete_data.join(results_data, on="athlete_id",rsuffix='right')
print(athleteResults['edition'])
print(olympic_games['edition'])
athleteResultsEdition = athleteResults.merge(olympic_games, on="edition")
print(athleteResultsEdition.columns)

0         2004 Summer Olympics
1         1948 Summer Olympics
2         1952 Summer Olympics
3         2004 Summer Olympics
4         1994 Winter Olympics
                  ...         
154896    2004 Summer Olympics
154897    2012 Summer Olympics
154898    2008 Summer Olympics
154899                     NaN
154900    2006 Winter Olympics
Name: edition, Length: 154901, dtype: object
0     1896 Summer Olympics
1     1900 Summer Olympics
2     1904 Summer Olympics
3     1908 Summer Olympics
4     1912 Summer Olympics
              ...         
57    2010 Winter Olympics
58    2014 Winter Olympics
59    2018 Winter Olympics
60    2022 Winter Olympics
61    2026 Winter Olympics
Name: edition, Length: 62, dtype: object
Index(['athlete_id', 'name', 'sex', 'born', 'height', 'weight', 'country',
       'country_noc_x', 'description', 'special_notes', 'edition',
       'country_nocright', 'sport', 'event', 'result_id', 'athlete',
       'athlete_idright', 'pos', 'medal', 'isTeamSport', 'edition

In [4]:
athleteResultsEdition['athleteCountry'] = 0
for idx,val in athleteResultsEdition.iterrows():
    if val['country_noc_x'] == val['country_noc_y']:
        athleteResultsEdition['athleteCountry'] == 1

In [5]:
athleteResultsEdition["edition"] = athleteResultsEdition["edition"].astype('category')
athleteResultsEdition["sex"] = athleteResultsEdition["sex"].astype('category')
athleteResultsEdition["country"] = athleteResultsEdition["country"].astype('category')
athleteResultsEdition["sport"] = athleteResultsEdition["sport"].astype('category')
athleteResultsEdition["event"] = athleteResultsEdition["event"].astype('category')
athleteResultsEdition["isTeamSport"] = athleteResultsEdition["isTeamSport"].astype('category')
athleteResultsEdition["medal"] = athleteResultsEdition["medal"].astype('category')
athleteResultsEdition["height"] = pd.to_numeric(athleteResultsEdition["height"],errors='coerce')
athleteResultsEdition["weight"] = pd.to_numeric(athleteResultsEdition["weight"],errors='coerce')

In [6]:
athleteResultsEdition["edition"] = athleteResultsEdition["edition"].cat.codes
athleteResultsEdition["sex"] = athleteResultsEdition["sex"].cat.codes
athleteResultsEdition["country"] = athleteResultsEdition["country"].cat.codes
athleteResultsEdition["sport"] = athleteResultsEdition["sport"].cat.codes
athleteResultsEdition["event"] = athleteResultsEdition["event"].cat.codes
athleteResultsEdition["isTeamSport"] = athleteResultsEdition["isTeamSport"].cat.codes
athleteResultsEdition["medal"] = athleteResultsEdition["medal"].cat.codes

In [7]:
athleteResultsFeatures = athleteResultsEdition[['sex','height','weight','country','edition','sport','event','isTeamSport','athleteCountry','medal']]

In [8]:
print(athleteResultsFeatures)

        sex  height  weight  country  edition  sport  event  isTeamSport  \
0         1     NaN     NaN      198       43     22    227            1   
1         1   178.0    60.0      533       43     84    725            1   
2         0   156.0    48.0      413       43      7    123            1   
3         0   159.0    50.0      283       43     46    370            1   
4         1   172.0    67.0      529       43     77     38            0   
...     ...     ...     ...      ...      ...    ...    ...          ...   
143071    1     NaN     NaN       14        9     56    387            0   
143072    1     NaN     NaN      313        9     73    164            0   
143073    1   177.0    77.0      283        9     24     63            0   
143074    0   154.0    48.0      535        9     47    377            1   
143075    1   196.0    91.0      505        9     18    317            1   

        athleteCountry  medal  
0                    0      3  
1                    0 

In [9]:
profile = ProfileReport(athleteResultsFeatures, title="Pandas Profiling Report")

In [10]:
profile.to_notebook_iframe()


Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

In [11]:
x = athleteResultsFeatures.copy()
y = x.pop('medal')
x = pd.get_dummies(x)
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.33)

In [12]:
imputer = KNNImputer(n_neighbors=2,copy=False)
X_train = imputer.fit_transform(X_train)

In [13]:
def confusion(label, pred, labelorder):
    label_cf = label
    unique = np.unique(label)
    i = 0
    for each in unique:
        label_cf = np.where(label_cf == unique[i], labelorder[i], label_cf)
        i+=1
    real = label_cf.tolist()
    cf = confusion_matrix(real,pred)
    print(cf)

In [14]:
import itertools

def label_permute_compare(ytdf,yp,y_uni):
    """
    ytdf: labels dataframe object
    yp: clustering label prediction output
    Returns permuted label order and accuracy. 
    Example output: (3, 4, 1, 2, 0), 0.74 
    """
    # your code here
    unique = np.unique(ytdf)
    perm = itertools.permutations(y_uni)
    accuracyMatrix = []
    for i in list(perm):
        j = 0
        ytdf_guess = ytdf
        for k in unique:
            ytdf_guess = np.where(ytdf_guess == k, i[j], ytdf_guess)
            j+=1
        pred = ytdf_guess.tolist()

        accuracy = accuracy_score(pred,yp)
        accuracyMatrix.append((i,accuracy))
    i = 0
    maxAccuracy = 0
    while i < len(accuracyMatrix):
        if accuracyMatrix[i][1] > maxAccuracy:
            maxtuple = accuracyMatrix[i]
            maxAccuracy = accuracyMatrix[i][1]
        i += 1
    
    print(maxtuple)
    return maxtuple

In [15]:

def kmeans(y_uni):
    kmeans = KMeans(n_clusters=len(y_uni))
    kmeans.fit(X_train)
    labelorder, acc = label_permute_compare(y_train, kmeans.labels_, y_uni)
    confusion(y_train, kmeans.labels_,labelorder)

In [16]:
y_uni = np.unique(y_train)

start = time.time_ns()
kmeans(y_uni)
end = time.time_ns()


((1, 2, 3, 0), 0.35766743167118714)
[[34286 27515 18888 15171]
 [    0     0     0     0]
 [    0     0     0     0]
 [    0     0     0     0]]
